In [1]:
# You should be able to just run the whole script to align
# Just run all cells
#this
import os
import yaml
import time
import numpy as np
import pandas as pd
import shelve
import sys
import random
import matplotlib.pyplot as plt
import scipy.stats as stats
from scipy.optimize import fsolve
from scipy.interpolate import interp1d
from scipy.optimize import minimize
from playsound import playsound

sys.path.append('../../')
from geecs_python_api.controls.interface import GeecsDatabase
from geecs_python_api.controls.devices.geecs_device import GeecsDevice
# from geecs_python_api.controls.experiment.experiment import Experiment



path config ..\..\..\..\..\..\user data\Configurations.INI
database name  loasis


In [19]:
#this

# This cell for BELLA LPMode setup
GeecsDevice.exp_info = GeecsDatabase.collect_exp_info("Bella")
camera = GeecsDevice('CAM-PL1-LPMode')
camera.subscribe_var_values(['Target.X', 'Target.Y', 'Target2.X', 'Target2.Y'])
exposure = camera.get('exposure')
print(f"Camera exposure is {exposure}")
print(camera.state)

picos = GeecsDevice('MCD-PL1-picosOFI1')
picos.subscribe_var_values(['Holy mirror horizontal', 'Holy mirror vertical'])
print("Initial pico motor positions:")
print(f"1: {picos.get('Position.Axis 3')}") # holy mirror horizontal
print(f"2: {picos.get('Position.Axis 4')}") # holy mirror vertical
print(picos.state)

hexapod = GeecsDevice('HEX-PL1-2')
hexapod.subscribe_var_values(['ypos', 'zpos']) # ypos for x, zpos for y
print("Initial hexapod positions:")
print(f"ypos: {hexapod.get('ypos')}") # axicon horizontal
print(f"zpos: {hexapod.get('zpos')}") # axicon vertical
print(hexapod.state)

stage = GeecsDevice('STAGE-PL1-LPmodeLong')
stage.subscribe_var_values(['position'])
print("Initial stage positions:")
print(f"ypos: {stage.get('position')}") # stage pos
print(stage.state)

Camera exposure is 0.005
{'fresh': True, 'shot number': None, 'GEECS device error': False, 'exposure': 0.005}
Initial pico motor positions:
1: -4043.0
2: 3111.0
{'fresh': True, 'shot number': None, 'GEECS device error': False, 'Holy mirror horizontal': -4043.0, 'Holy mirror vertical': 3111.0}
Initial hexapod positions:
ypos: 0.8
zpos: -2.8364
{'fresh': True, 'shot number': 0, 'GEECS device error': False, 'ypos': 0.8, 'Device Status': 'Initialized', '': '', 'device error': '', 'zpos': -2.8364}
Initial stage positions:
ypos: 564.999995
{'fresh': True, 'shot number': None, 'GEECS device error': False, 'position': 564.999995}


In [13]:


#this

steps_per_pixel_picos_x = 5.5555 # at stage pos 570
# at 550: -5.1546
# ratio: 1.07777519109
# This means stage pos 570 is 257.15140933mm away from axis of rotation
steps_per_pixel_picos_y = 5.6818
steps_per_pixel_conex_x = -0.0001201923
steps_per_pixel_conex_y = 0.00016286644
steps_per_pixel_hex_x = 6.5e-4
steps_per_pixel_hex_y = -6.5e-4

mcd_dict = {
    'HM_x': { # Holy Mirror pico
        'device': picos, 
        'name': 'Position.Axis 3', 
        'alias': 'Holy mirror horizontal', 
        'calib': steps_per_pixel_picos_x, 
        'polyfit': None, 
        'polyfitfine': None, 
        'axis': 'x',
        'min': picos.get('Position.Axis 3') - 100000, # Software limits for beampointing
        'max': picos.get('Position.Axis 3') + 100000, # You can change the +- to be larger if the beam doesn't fit
       },
    'HM_y': { # Holy mirror pico
        'device': picos, 
        'name': 'Position.Axis 4', 
        'alias': 'Holy mirror vertical', 
        'calib': steps_per_pixel_picos_y, 
        'polyfit': None, 
        'polyfitfine': None, 
        'axis': 'y', 
        'min': picos.get('Position.Axis 4') - 100000,
        'max': picos.get('Position.Axis 4') + 100000,
       },
#     'HM_x': { # Holy mirror x
#         'device': conex_x, 
#         'name': 'position', 
#         'alias': 'position', 
#         'calib': steps_per_pixel_conex_x, 
#         'polyfit': None, 
#         'polyfitfine': None, 
#         'axis': 'x',
#         'min': 0,
#         'max': 10,
#     },
#     'HM_y': { # Holy mirror y
#         'device': conex_y, 
#         'name': 'position', 
#         'alias': 'position', 
#         'calib': steps_per_pixel_conex_y, 
#         'polyfit': None, 
#         'polyfitfine': None, 
#         'axis': 'x',
#         'min': 0,
#         'max': 10,
#     },
    'axicon_x': {
        'device': hexapod, 
        'name': 'ypos', 
        'alias': 'ypos', 
        'calib': steps_per_pixel_hex_x, 
        'polyfit': None, 
        'polyfitfine': None, 
        'axis': 'x',
        'min': hexapod.get('ypos') - 0.4,
        'max': hexapod.get('ypos') + 0.4,
    },
    'axicon_y': {
        'device': hexapod, 
        'name': 'zpos', 
        'alias': 'zpos', 
        'calib': steps_per_pixel_hex_y, 
        'polyfit': None, 
        'polyfitfine': None, 
        'axis': 'y',
        'min': hexapod.get('zpos') - 0.4,
        'max': hexapod.get('zpos') + 0.4,
    },
}

In [6]:
#this
# Hyperparameters
SHOTS_TO_AVG = 3
TIME_BETWEEN_SHOTS = 0.1 # seconds

In [7]:
def set_with_limit(mcd, value, min_val=None, max_val=None):
    """
    Set a parameter with software limits, default is no limit
    This is a blocking function since we are setting
    
    mcd: device dictionary in mcd_dict
    value: value to set to
    min_val, max_val: you can temporarily override the software limits by specifying
    a non-null value
    
    return: actual value set
    """
    if not min_val:
        min_val = mcd['min']
    if not max_val:
        max_val = mcd['max']
        
    if min_val > max_val:
        raise Exception('min_val must be smaller than max_val')
        
    mcd['device'].set(mcd['name'], max(min_val, min(max_val, value)))
    return max(min_val, min(max_val, value))

In [8]:
#this
# Beam pointing onto target
def obj_func(targetx, targety, beamx, beamy):
    return (targetx - beamx)**2 + (targety - beamy)**2

def move_random(pico_num, picos, dmin, dmax):
    """
    pico_num: an index for mcd_dict
    picos: GEECS picos object
    moves the pico between [dmin, dmax] randomly from its original position
    """
    picomotor = mcd_dict[pico_num]
    new_pos = picos.state[picomotor['name']] + random.randint(dmin, dmax)
    print(f"moving motor {pico_num} to position {new_pos}")
    picos.set(picomotor['name'], int(picos.state[picomotor['name']] + random.randint(dmin, dmax)))
    return

def get_centroid(camera, num_shots, time_between_shots):
    """
    Returns (x,y) of centroid of beam
    """
    centroidx = []
    centroidy = []
    # Average over SHOTS_TO_AVG shots
    for i in range(num_shots):
        # While loops since sometimes the camera doesn't send back a value for centroid
        centroidx1, centroidy1 = None, None
        while not isinstance(centroidx1, float):
            centroidx1 = camera.get('Centroidx')
#             print(centroidx1)
        centroidx.append(centroidx1)
        while not isinstance(centroidy1, float):
            centroidy1 = camera.get('Centroidy')
        centroidy.append(centroidy1)
        print(f"Centroid is {centroidx1}, {centroidy1}")
        time.sleep(time_between_shots)

    centroidx = np.mean(np.array(centroidx))
    centroidy = np.mean(np.array(centroidy))
    return centroidx, centroidy

def move_onto_target(camera, obj_target, opt_rate, pico1, pico2, target, fit='linear', 
                     fine_thresh=10, hexapod_flag=True, verbose=True):
    """
    Moves a beam onto a target with pico1 and pico2
    
    camera: GEECS camera object
    obj_target: stop optimizing once obj_func < obj_target
    opt_rate: we use a linear approximation of steps per pixel
    to approximate where the target should be, but multiply by opt_rate
    pico1, pico2: choose 2 picos, they must be one in the x direction and one
    in the y direction, these should be keys in the mcd_dict
    target: an integer 1 or 2, this is which target we are pointing at
    fit: if linear, use linear calib (steps_per_pixel)
    if poly, use polynomial fit (these should be PolynomialModel instances inside mcd_dict)
    hexapod_flag: True if aligning hexapods, False if aligning picos
    (This is because hexapods take floats as inputs, and picos take ints as inputs)
    
    returns: iterations, obj_func()
    """
    if hexapod_flag:
        print("Moving a hexapod")
    else:
        print("Moving pico motors")
    
    # Helper function to calculate objective function
    def obj():
        centroidx, centroidy = get_centroid(camera, SHOTS_TO_AVG, TIME_BETWEEN_SHOTS)
            
        if target == 1:
            return obj_func(camera.state['Target.X'], camera.state['Target.Y'], 
                            centroidx, centroidy)
        elif target == 2:
            return obj_func(camera.state['Target2.X'], camera.state['Target2.Y'], 
                            centroidx, centroidy)
        else:
            raise Exception('target must be either 1 or 2')
    
    # Helper function to get target position
    def target_pos():
        if target == 1:
            return camera.state['Target.X'], camera.state['Target.Y']
        elif target == 2:
            return camera.state['Target2.X'], camera.state['Target2.Y']
        else:
            raise Exception('target must be either 1 or 2')
            
    # giving mcds more convenient names
    if mcd_dict[pico1]['axis'] == 'x' and mcd_dict[pico2]['axis'] == 'y':
        mcd_x = mcd_dict[pico1] # picomotor means the motor dictionary entry
        mcd_y = mcd_dict[pico2]
    elif mcd_dict[pico1]['axis'] == 'y' and mcd_dict[pico2]['axis'] == 'x':
        mcd_x = mcd_dict[pico2]
        mcd_y = mcd_dict[pico1]
    else:
        raise Exception('PICOs must span both x and y axes')
        
    # Move mcds until beam is close enough to target
    iterations = 0
    target_x, target_y = target_pos()
    while obj() > obj_target:
        iterations += 1
        if verbose:
            print(f"Iteration {iterations}")
        pico_x = mcd_x['device'].state[mcd_x['alias']]
        pico_y = mcd_y['device'].state[mcd_y['alias']]
        if verbose:
            print(f"Original pico position is ({int(pico_x)}, {int(pico_y)})")
        centroidx = None
        centroidy = None
        # While loops since sometimes the camera doesn't send back a value for centroid
        while not isinstance(centroidx, float):
            centroidx = camera.get('Centroidx')
        while not isinstance(centroidy, float):
            centroidy = camera.get('Centroidy')
        dx = target_x - centroidx
        dy = target_y - centroidy
        if verbose:
            print(f"original centroid position = ({centroidx}, {centroidy})")
            print(f"dx = {dx}, dy = {dy}")
        if fit == "linear":
            move_x = (dx) * mcd_x['calib'] * opt_rate
            move_y = (dy) * mcd_y['calib'] * opt_rate
        if fit == "poly":
            if np.abs(dx) > fine_thresh:
                move_x = mcd_x['polyfit'].predict_motor_move(dx) * opt_rate
            else:
                move_x = mcd_x['polyfitfine'].predict_motor_move(dx) * opt_rate
            if np.abs(dy) > fine_thresh:
                move_y = mcd_y['polyfit'].predict_motor_move(dy) * opt_rate
            else:
                move_y = mcd_y['polyfitfine'].predict_motor_move(dy) * opt_rate
        if verbose:
            if hexapod_flag:
                print(f"Moving by ({move_x}, {move_y})")
            else:
                print(f"Moving by ({int(move_x)}, {int(move_y)})")
            print(f"Moving to ({pico_x + move_x}, {pico_y + move_y})")
        if hexapod_flag: # hexapods take float moves
            set_with_limit(mcd_x, np.round(pico_x + move_x, 5))
            set_with_limit(mcd_y, np.round(pico_y + move_y, 5))
            if verbose:
                print(f"Final position is ({mcd_x['device'].state[mcd_x['alias']]}, "
                      f"{mcd_x['device'].state[mcd_y['alias']]})")
        else: # picos take int moves
            set_with_limit(mcd_x, int(pico_x + move_x))
            set_with_limit(mcd_y, int(pico_y + move_y))
            if verbose:
                print(f"Final position is ({int(mcd_x['device'].state[mcd_x['alias']])}, "
                      f"{int(mcd_x['device'].state[mcd_y['alias']])})")
        time.sleep(0.1)
        
    
    return iterations, obj()

def move_onto_target_x(camera, obj_target, opt_rate, mcd1, target):
    """
    Moves a beam onto a target with pico1 and pico2
    
    camera: GEECS camera object
    obj_target: stop optimizing once obj_func < obj_target
    opt_rate: we use a linear approximation of steps per pixel
    to approximate where the target should be, but multiply by opt_rate
    pico1, pico2: choose 2 picos, they must be one in the x direction and one
    in the y direction, these should be keys in the mcd_dict
    target: an integer 1 or 2, this is which target we are pointing at
    fit: if linear, use linear calib (steps_per_pixel)
    if poly, use polynomial fit (these should be PolynomialModel instances inside mcd_dict)
    hexapod_flag: True if aligning hexapods, False if aligning picos
    (This is because hexapods take floats as inputs, and picos take ints as inputs)
    
    returns: iterations, obj_func()
    """
    
    # Helper function to calculate objective function
    def obj():
        centroidx, centroidy = get_centroid(camera, SHOTS_TO_AVG, TIME_BETWEEN_SHOTS)
            
        if target == 1:
            return (camera.state['Target.X'] - centroidx)**2
        elif target == 2:
            return (camera.state['Target2.X'] - centroidx)**2
        else:
            raise Exception('target must be either 1 or 2')
    
    # Helper function to get target position
    def target_pos():
        if target == 1:
            return camera.state['Target.X'], camera.state['Target.Y']
        elif target == 2:
            return camera.state['Target2.X'], camera.state['Target2.Y']
        else:
            raise Exception('target must be either 1 or 2')
    
    mcd_x = mcd_dict[mcd1]
        
    # Move mcds until beam is close enough to target
    iterations = 0
    target_x, target_y = target_pos()
    while obj() > obj_target:
        iterations += 1
        print(f"Iteration {iterations}")
        pico_x = mcd_x['device'].state[mcd_x['alias']]
        print(f"Original pico position is ({float(pico_x)})")
        centroidx = None
        # While loops since sometimes the camera doesn't send back a value for centroid
        while not isinstance(centroidx, float):
            centroidx = camera.get('Centroidx')
        dx = target_x - centroidx
        print(f"original centroid position = ({centroidx})")
        print(f"dx = {dx}")
        move_x = (dx) * mcd_x['calib'] * opt_rate
        conex_move_hysteresis(mcd1, np.round(pico_x + move_x, 5), CONEX_HYSTERESIS_SIZE)
        
        print(f"Final position is ({mcd_x['device'].state[mcd_x['alias']]})")
        time.sleep(0.1)
        
    
    return iterations, obj()
    
    

In [9]:
def move_pico_to_pixel(camera, obj_target, opt_rate, pico_x, pico_y, target_pix_x, target_pix_y):
    """
    Moves a beam onto a target with pico1 and pico2
    
    camera: GEECS camera object
    obj_target: stop optimizing when we are less than obj_target pixels in the given axis
    opt_rate: we use a linear approximation of steps per pixel
    to approximate where the target should be, but multiply by opt_rate
    pico_x, pico_y: object in the mcd dict, this should be a picomotor
    target_pix_x, target_pix_y: pixel to move to
    
    returns: iterations, obj_func()
    """
    
    # Helper function to get target position
    def obj():
        centroidx, centroidy = get_centroid(camera, SHOTS_TO_AVG, TIME_BETWEEN_SHOTS)
        obj = (target_pix_x - centroidx)**2 + (target_pix_y - centroidy)**2
        print(f"In move pico: Objective is {obj}")
        return obj
        
    print(f"Target pixel is: (x,y) = ({target_pix_x}, {target_pix_y})")
    # Move mcds until beam is close enough to target
    iterations = 0
    while obj() > obj_target:
        iterations += 1
        print(f"Iteration {iterations}")
        pico_pos_x = pico_x['device'].state[pico_x['alias']]
        pico_pos_y = pico_y['device'].state[pico_y['alias']]
        print(f"Original pico position is ({pico_pos_x}, {pico_pos_y})")
        centroidx, centroidy = get_centroid(camera, SHOTS_TO_AVG, TIME_BETWEEN_SHOTS)
        # While loops since sometimes the camera doesn't send back a value for centroid
        while not isinstance(centroidx, float):
            centroidx, centroidy = get_centroid(camera, SHOTS_TO_AVG, TIME_BETWEEN_SHOTS)
        dx = target_pix_x - centroidx
        dy = target_pix_y - centroidy
        print(f"original centroid position = ({centroidx}, {centroidy})")
        print(f"dx = {dx}, dy = {dy}")
        move_x = dx * pico_x['calib'] * opt_rate
        move_y = dy * pico_y['calib'] * opt_rate
        set_with_limit(pico_x, int(pico_pos_x + move_x))
        set_with_limit(pico_y, int(pico_pos_y + move_y))
        
        print(f"Final position is picox = ({pico_x['device'].state[pico_x['alias']]}), "
              f"picoy = ({pico_y['device'].state[pico_y['alias']]})")
        time.sleep(1)
        
    
    return iterations, obj()

def move_axicon_to_pixel(camera, obj_target, opt_rate, axicon_x, axicon_y, target_pix_x, target_pix_y):
    """
    Moves a beam onto a target with axicon1 and axicon2
    
    camera: GEECS camera object
    obj_target: stop optimizing when we are less than obj_target pixels in the given axis
    opt_rate: we use a linear approximation of steps per pixel
    to approximate where the target should be, but multiply by opt_rate
    axicon_x, axicon_y: object in the mcd dict, this should be a axicon direction
    target_pix_x, target_pix_y: pixel to move to
    
    returns: iterations, obj_func()
    """
    
    # Helper function to get target position
    def obj():
        centroidx, centroidy = get_centroid(camera, SHOTS_TO_AVG, TIME_BETWEEN_SHOTS)
        obj = (target_pix_x - centroidx)**2 + (target_pix_y - centroidy)**2
        print(f"Obj in move axicon is {obj}")
        return obj
        
    centroidx = None
    print(f"Target pixel is: (x,y) = ({target_pix_x}, {target_pix_y})")
    # Move mcds until beam is close enough to target
    iterations = 0
    while obj() > obj_target:
        iterations += 1
        print(f"Iteration {iterations}")
        axicon_pos_x = axicon_x['device'].state[axicon_x['alias']]
        axicon_pos_y = axicon_y['device'].state[axicon_y['alias']]
        print(f"Original axicon position is ({axicon_pos_x}, {axicon_pos_y})")
        centroidx, centroidy = get_centroid(camera, SHOTS_TO_AVG, TIME_BETWEEN_SHOTS)
        # While loops since sometimes the camera doesn't send back a value for centroid
        while not isinstance(centroidx, float):
            centroidx, centroidy = get_centroid(camera, SHOTS_TO_AVG, TIME_BETWEEN_SHOTS)
        dx = target_pix_x - centroidx
        dy = target_pix_y - centroidy
        print(f"original centroid position = ({centroidx}, {centroidy})")
        print(f"dx = {dx}, dy = {dy}")
        move_x = dx * axicon_x['calib'] * opt_rate
        move_y = dy * axicon_y['calib'] * opt_rate
        set_with_limit(axicon_x, axicon_pos_x + move_x)
        set_with_limit(axicon_y, axicon_pos_y + move_y)
        
        print(f"Final position is axiconx = ({axicon_x['device'].state[axicon_x['alias']]}), "
              f"axicony = ({axicon_y['device'].state[axicon_y['alias']]})")
        time.sleep(1)
        
    
    return iterations, obj()
    

In [10]:
def align_2_targets_at_once(mcd_1x, mcd_1y, mcd_2x, mcd_2y, camera, obj_target, 
                            opt_rate, stage_pos_a, stage_pos_b, lambda_1, lambda_2):
    """
    Uses 2 different calibrations to align to 2 targets at once
    When using this function, make sure that "Target.X" and "Target.Y" are the upstream camera
    position and "Target2.X"/"Target2.Y" are the downstream position
    
    mcd_1x, mcd_1y: dictionary entries for upstream mcd x and y controllers
    mcd_2x, mcd_2y: dictionary entries for downstream mcd x and y controllers
    It is assumed that the upstream mcd "aligns to" the closer position, while the downstream mcd
    "aligns to" the further position in traditional alignment
    camera: GEECS camera object
    picos: GEECS picos object
    obj_target: stop optimizing once obj_func < obj_target
    opt_rate: we use a linear approximation of steps per pixel
    to approximate where the target should be, but multiply by opt_rate
    stage_pos_a: closer numerical value for stage position
    stage_pos_b: far numerical value for stage position
    STAGE_POS1 SHOULD BE FURTHER DOWNSTREAM THAN STAGE_POS2
    lambda_1: ratio of pixels moved of stage_pos_b/stage_pos_a for mcd_1
    lambda_2: ratio of pixels moved of stage_pos_b/stage_pos_a for mcd_2
    """
    #Delta x_a = move_1a + move_2a
    #Delta x_b = lambda_1*move_1a + lambda_2*move_2a
    #move_1a means the number of pixels to move on mcd 1
    begin_time = time.time()
    
    while True:
        t1 = time.time()
        stage.set('position', stage_pos_a)
        print(f"Time for stage to move: {time.time() - t1}")
        time.sleep(3)
        print(f"Time after sleep: {time.time() - t1}")
        centroid_close_x, centroid_close_y = get_centroid(camera, SHOTS_TO_AVG, TIME_BETWEEN_SHOTS)
        t1 = time.time()
        stage.set('position', stage_pos_b)
        print(f"Time for stage to move: {time.time() - t1}")
        time.sleep(3)
        print(f"Time after sleep: {time.time() - t1}")
        centroid_far_x, centroid_far_y = get_centroid(camera, SHOTS_TO_AVG, TIME_BETWEEN_SHOTS)

        # X matrix equation (A * \vec{pos} = \vec{b})
        # A is matrix of calibrations
        # pos = positions/values of mcds
        # b = pixel difference in target vs centroid
        # Must do this for x and y direction

        # x-dir
        A = np.array([
            [1, 1],
            [lambda_1, lambda_2],
        ])
#         print("Matrix A is (X-dir):", A)
        b = np.array([
            [camera.state['Target.X'] - centroid_close_x],
            [camera.state['Target2.X'] - centroid_far_x],
        ])
        pixel_moves_x = np.matmul(np.linalg.inv(A), b).flatten()
        pixel_moves_x *= [lambda_1, lambda_2]
        
        # y-dir
        A = np.array([
            [1, 1],
            [lambda_1, lambda_2],
        ])
#         print("Matrix A is (X-dir):", A)
        b = np.array([
            [camera.state['Target.Y'] - centroid_close_y],
            [camera.state['Target2.Y'] - centroid_far_y],
        ])
        pixel_moves_y = np.matmul(np.linalg.inv(A), b).flatten()
        pixel_moves_y *= [lambda_1, lambda_2]
        
#         print(f"b = {b}")
        print(f"pixel moves x: {pixel_moves_x}")
        print(f"pixel moves y: {pixel_moves_y}")
        # move 
        move_axicon_to_pixel(camera, obj_target**2, 1, mcd_1x, mcd_1y, 
                             centroid_far_x + pixel_moves_x[0],
                             centroid_far_y + pixel_moves_y[0])
        move_pico_to_pixel(camera, obj_target**2/4, opt_rate, mcd_2x, mcd_2y, 
                           centroid_far_x + pixel_moves_x[0] + pixel_moves_x[1], 
                           centroid_far_y + pixel_moves_y[0] + pixel_moves_y[1])
        
        # Check to see if moved to correct positions
        time.sleep(2)
        centroid_far_x, centroid_far_y = get_centroid(camera, SHOTS_TO_AVG, TIME_BETWEEN_SHOTS)
        stage.set('position', stage_pos_a)
        time.sleep(3)
        centroid_close_x, centroid_close_y = get_centroid(camera, SHOTS_TO_AVG, TIME_BETWEEN_SHOTS)
        
        obj1 = np.abs(centroid_close_x - camera.state['Target.X'])
        obj2 = np.abs(centroid_far_x - camera.state['Target2.X'])
        
        print(f"\nObj 1 = {obj1}, Obj2 = {obj2}\n")
        
        if (obj1 < obj_target) and (obj2 < obj_target):
            break
    
    stage.set('position', stage_pos_a)
    time.sleep(3)
    centroid1_x, centroid1_y = get_centroid(camera, SHOTS_TO_AVG, TIME_BETWEEN_SHOTS)
    stage.set('position', stage_pos_b)
    time.sleep(3)
    centroid2_x, centroid2_y = get_centroid(camera, SHOTS_TO_AVG, TIME_BETWEEN_SHOTS)
    print("Close delta x:", centroid1_x - camera.state['Target.X'])
    print("Close delta y:", centroid1_y - camera.state['Target.Y'])
    print("Far delta x:", centroid2_x - camera.state['Target2.X'])
    print("Far delta y:", centroid2_y - camera.state['Target2.X'])
    print(f"Alignment took {time.time() - begin_time} seconds")
    
    playsound("applause.mp3")
    
    return
    
    
    

In [15]:
stage_pos_upstream = 580
stage_pos_downstream = 565
ratio = (827 - stage_pos_downstream) / (827 - stage_pos_upstream)
assert(ratio > 1)

In [20]:
align_2_targets_at_once(mcd_dict['axicon_x'], mcd_dict['axicon_y'], mcd_dict['HM_x'], mcd_dict['HM_y'], camera, 4, 
                            0.8, stage_pos_upstream, stage_pos_downstream, 1, ratio)

Time for stage to move: 2.024611473083496
Time after sleep: 5.025623798370361
Centroid is 633.149658, 563.988708
Centroid is 632.194641, 562.741699
Centroid is 632.504211, 560.875549
Time for stage to move: 2.2051284313201904
Time after sleep: 5.206140995025635
Centroid is 611.341675, 566.675415
Centroid is 611.437683, 566.365356
Centroid is 611.696838, 566.056885
pixel moves x: [ 90.67357316 -85.16563849]
pixel moves y: [133.00801244 -84.37389778]
Target pixel is: (x,y) = (702.1656384888871, 699.3738977777773)
Centroid is 612.946167, 565.004211
Centroid is 614.430603, 567.111145
Centroid is 609.918213, 567.210754
Obj in move axicon is 25723.066414439705
Iteration 1
Original axicon position is (0.8, -2.8364)
Centroid is 613.170166, 565.109985
Centroid is 612.793152, 566.448975
Centroid is 614.314026, 564.618164
original centroid position = (613.4257813333334, 565.3923746666667)
dx = 88.73985715555375, dy = 133.98152311111062
Final position is axiconx = (0.8577), axicony = (-2.923488)
C

PlaysoundException: 
    Error 275 for command:
        open "applause.mp3" alias playsound_0.3273641794237421
    Cannot find the specified file.  Make sure the path and filename are correct.

In [21]:
camera.close()
picos.close()
hexapod.close()
stage.close()